In [1]:
import findspark

In [2]:
findspark.init('/home/joe/spark-2.4.3-bin-hadoop2.7')

In [3]:
import pyspark

In [4]:
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder.master("local").appName('health').getOrCreate()

In [6]:
df = spark.read.json('data/healthcare04')

In [7]:
df.printSchema()

root
 |-- _corrupt_record: string (nullable = true)
 |-- contributors: string (nullable = true)
 |-- coordinates: struct (nullable = true)
 |    |-- coordinates: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |    |-- type: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- display_text_range: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- entities: struct (nullable = true)
 |    |-- hashtags: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- indices: array (nullable = true)
 |    |    |    |    |-- element: long (containsNull = true)
 |    |    |    |-- text: string (nullable = true)
 |    |-- media: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- display_url: string (nullable = true)
 |    |    |    |-- expanded_url: string (nullable = true)
 |    |    |    |-- id: long (nullable = true)
 |    |    |    |-- id_

In [8]:
df.head()

Row(_corrupt_record=None, contributors=None, coordinates=None, created_at='Sun Jul 30 03:51:49 +0000 2017', display_text_range=[0, 140], entities=Row(hashtags=[], media=None, symbols=[], urls=[], user_mentions=[Row(id=800942537083068400, id_str='800942537083068416', indices=[3, 15], name="Women's March", screen_name='womensmarch'), Row(id=19726613, id_str='19726613', indices=[18, 33], name='Sen. Susan Collins', screen_name='SenatorCollins'), Row(id=18061669, id_str='18061669', indices=[38, 52], name='Sen. Lisa Murkowski', screen_name='lisamurkowski')]), extended_entities=None, favorite_count=0, favorited=False, full_text='RT @womensmarch: .@SenatorCollins and @lisamurkowski voted for women and families last night. They stayed strong from the beginning. https:…', geo=None, id=891506592977346600, id_str='891506592977346560', in_reply_to_screen_name=None, in_reply_to_status_id=None, in_reply_to_status_id_str=None, in_reply_to_user_id=None, in_reply_to_user_id_str=None, is_quote_status=Fal

In [9]:
df.createOrReplaceTempView("health")

In [10]:
df1 = spark.sql("SELECT DISTINCT created_at AS date,user.id AS user_id,user.name AS user_name,favorite_count,retweet_count,full_text AS text,user.location AS user_location FROM health WHERE lang = 'en' AND user.location REGEXP 'Newfoundland and Labrador|, NL$|Alberta|, AB$|Saskatchewan|, SK$|Prince Edward Island|, PE$|British Columbia|, BC$|Manitoba|, MB$|New Brunswick|, NB$|Nova Scotia|, NS$|Quebec|, QC$|Ontario|, ON$'")

In [2]:
df1.count()

NameError: name 'df1' is not defined

In [12]:
df1.head()

Row(date='Sun Jul 30 07:45:31 +0000 2017', user_id=380664031, user_name='James Toupin', favorite_count=0, retweet_count=7845, text="RT @TheRickyDavila: MSNBC's @AliVelshi calls out EVERY Senator who were paid off by the insurance companies. This is disgusting. SHARE THIS…", user_location='Edmonton, Alberta, Canada')

In [11]:
#from pyspark.sql.functions import col, concat

In [ ]:
#df1.withColumn('tokens_bigrams', concat(col('tokens'), col('bigrams')))

In [10]:
#df1.withColumn("name", concat_ws(',','name')).show()

NameError: name 'concat_ws' is not defined

In [9]:
#from pyspark.sql.functions import udf
#from pyspark.sql.types import StringType

In [10]:
#def array_to_string(my_list):
#    return  ','.join([str(elem) for elem in my_list])

In [11]:
#array_to_string_udf = udf(array_to_string,StringType())

In [12]:
#df2 = df1.withColumn('name',array_to_string_udf(df1["name"]))

In [13]:
df1.show()

+--------------------+------------------+--------------------+--------------+-------------+--------------------+--------------------+
|                date|           user_id|           user_name|favorite_count|retweet_count|                text|       user_location|
+--------------------+------------------+--------------------+--------------+-------------+--------------------+--------------------+
|Sun Jul 30 07:45:...|         380664031|        James Toupin|             0|         7845|RT @TheRickyDavil...|Edmonton, Alberta...|
|Sun Jul 30 14:31:...|         293774123|    Jan Ravensbergen|             0|          600|RT @mattmfm: A we...|Montreal, Quebec,...|
|Sun Jul 30 14:55:...|         127309627|           Paul Rose|             0|            0|Riddle me this - ...|    Ontario - Canada|
|Sun Jul 30 15:19:...|         126498319|   Peggy S. Mitchell|             0|          429|RT @ThePlumLineGS...|Edmonton, Alberta...|
|Sun Jul 30 16:02:...|789512359693938700|         Mike Almond|

In [14]:
df1.coalesce(1).write.mode("overwrite").csv('part04')